# 👥 Multi-Agent - 여러 에이전트 협업

이 노트북은 beanllm의 Multi-Agent 시스템을 학습합니다. 여러 Agent가 협업하여 복잡한 작업을 수행할 수 있습니다.

## 📋 목차

- [1. Multi-Agent 기본](#1-multi-agent-기본)
- [2. 순차 협업 (Sequential)](#2-순차-협업-sequential)
- [3. 병렬 협업 (Parallel)](#3-병렬-협업-parallel)
- [4. 계층적 구조](#4-계층적-구조)
- [5. 에이전트 간 통신](#5-에이전트-간-통신)
- [6. 실전 예제](#6-실전-예제)

---

## 🎯 학습 목표

이 노트북을 완료하면:
- ✅ 여러 Agent를 생성하고 관리할 수 있습니다
- ✅ 순차적 및 병렬 협업을 구성할 수 있습니다
- ✅ 에이전트 간 메시지 전달을 이해할 수 있습니다
- ✅ 복잡한 Multi-Agent 시스템을 구축할 수 있습니다
- ✅ 조정 전략을 선택할 수 있습니다

---

## 📚 사전 요구사항

- [01_setup_and_installation.ipynb](01_setup_and_installation.ipynb) 완료
- [02_core_client.ipynb](02_core_client.ipynb) 완료
- [05_agent_and_tools.ipynb](05_agent_and_tools.ipynb) 완료


## 1. Multi-Agent 기본

Multi-Agent 시스템은 여러 Agent가 협업하여 단일 Agent로는 불가능한 복잡한 작업을 수행합니다.

> 💡 **Multi-Agent의 핵심**:
> - 각 Agent는 특정 역할과 전문성을 가집니다
> - Agent 간 통신을 통해 정보를 공유합니다
> - 조정 전략에 따라 순차 또는 병렬로 실행됩니다


In [ ]:
import asyncio
from beanllm import Agent, MultiAgentCoordinator


async def basic_multi_agent():
    """기본 Multi-Agent 예제"""

    # 여러 Agent 생성
    researcher = Agent(
        model="gpt-4o",
        name="researcher",
        system_prompt="당신은 연구원입니다. 정보를 수집하고 분석합니다.",
    )

    writer = Agent(
        model="gpt-4o",
        name="writer",
        system_prompt="당신은 작가입니다. 연구 결과를 바탕으로 글을 작성합니다.",
    )

    # Coordinator 생성
    coordinator = MultiAgentCoordinator(agents={"researcher": researcher, "writer": writer})

    # 순차 실행
    result = await coordinator.execute_sequential(
        task="AI의 미래에 대해 연구하고 글을 작성해주세요.", agent_order=["researcher", "writer"]
    )

    print("✅ Multi-Agent 실행 완료!")
    print(f"📝 최종 결과: {result['final_result'][:200]}...")
    print(f"\n📊 중간 결과 수: {len(result.get('intermediate_results', []))}개")

    return result


# 실행 (주석 해제)
# result = await basic_multi_agent()
print("💡 위의 주석을 해제하여 실행하세요.")

## 2. 순차 협업 (Sequential)

Agent들이 순서대로 실행되며, 이전 Agent의 출력이 다음 Agent의 입력이 됩니다.

### 2.1 연구-작성-편집 파이프라인

실제 사용 사례: 연구 → 작성 → 편집


In [ ]:
async def research_write_edit_pipeline():
    """연구-작성-편집 파이프라인"""

    # 1. 연구원 Agent
    researcher = Agent(
        model="gpt-4o",
        name="researcher",
        system_prompt="당신은 연구원입니다. 주제에 대한 핵심 사실을 수집합니다.",
    )

    # 2. 작가 Agent
    writer = Agent(
        model="gpt-4o",
        name="writer",
        system_prompt="당신은 작가입니다. 연구 결과를 바탕으로 명확한 글을 작성합니다.",
    )

    # 3. 편집자 Agent
    editor = Agent(
        model="gpt-4o",
        name="editor",
        system_prompt="당신은 편집자입니다. 글을 다듬고 오류를 수정합니다.",
    )

    # Coordinator 생성
    coordinator = MultiAgentCoordinator(
        agents={"researcher": researcher, "writer": writer, "editor": editor}
    )

    # 순차 실행
    task = "운동의 이점에 대해 글을 작성해주세요."

    result = await coordinator.execute_sequential(
        task=task, agent_order=["researcher", "writer", "editor"]
    )

    print("📚 연구-작성-편집 파이프라인 실행 완료!\n")
    print("=" * 60)
    print("중간 결과:")
    print("=" * 60)

    for i, (agent_name, answer) in enumerate(
        zip(["researcher", "writer", "editor"], result.get("intermediate_results", [])), 1
    ):
        print(f"\n[{agent_name}]:")
        print(answer[:200] + "..." if len(answer) > 200 else answer)

    print("\n" + "=" * 60)
    print("최종 결과:")
    print("=" * 60)
    print(result["final_result"])


# 실행 (주석 해제)
# await research_write_edit_pipeline()
print("💡 위의 주석을 해제하여 실행하세요.")

## 3. 병렬 협업 (Parallel)

여러 Agent가 동시에 실행되어 빠른 결과를 얻을 수 있습니다.

### 3.1 투표 기반 합의

여러 Agent의 답변을 투표로 합의합니다.


In [ ]:
async def parallel_voting():
    """병렬 실행 및 투표"""

    # 같은 모델로 여러 Agent 생성
    agents = {
        f"expert_{i}": Agent(
            model="gpt-4o",
            name=f"expert_{i}",
            system_prompt=f"당신은 전문가 {i+1}입니다. 간결하게 답변하세요.",
        )
        for i in range(3)
    }

    coordinator = MultiAgentCoordinator(agents=agents)

    # 병렬 실행 및 투표
    task = "프랑스의 수도는 어디인가요? 도시 이름만 답하세요."

    result = await coordinator.execute_parallel(task=task, aggregation="vote")  # 투표 방식

    print("📊 병렬 실행 및 투표 결과:\n")
    print("=" * 60)
    print("모든 답변:")
    for i, answer in enumerate(result.get("all_answers", []), 1):
        print(f"  {i}. {answer}")

    print(f"\n투표 결과: {result.get('vote_counts', {})}")
    print(f"최종 결과 (투표): {result.get('final_result', 'N/A')}")
    print(f"합의율: {result.get('agreement_rate', 0):.1%}")


# 실행 (주석 해제)
# await parallel_voting()
print("💡 위의 주석을 해제하여 실행하세요.")

### 3.2 합의 (Consensus) 모드

모든 Agent가 동의할 때까지 반복합니다.


In [ ]:
async def consensus_mode():
    """합의 모드 예제"""

    agents = {
        f"agent_{i}": Agent(
            model="gpt-4o", name=f"agent_{i}", system_prompt=f"당신은 에이전트 {i+1}입니다."
        )
        for i in range(3)
    }

    coordinator = MultiAgentCoordinator(agents=agents)

    # 합의 모드 실행
    result = await coordinator.execute_parallel(
        task="Python의 주요 특징 3가지를 나열하세요.", aggregation="consensus"  # 합의 방식
    )

    print("🤝 합의 모드 결과:\n")
    print(f"합의 도달: {result.get('consensus', False)}")
    if result.get("consensus"):
        print(f"합의된 답변: {result.get('final_result', 'N/A')}")
    else:
        print("합의에 도달하지 못했습니다.")


# 실행 (주석 해제)
# await consensus_mode()
print("💡 위의 주석을 해제하여 실행하세요.")

## 4. 계층적 구조

관리자 Agent가 여러 작업자 Agent를 조정하는 구조입니다.


In [ ]:
async def hierarchical_structure():
    """계층적 구조 예제"""

    # 관리자 Agent
    manager = Agent(
        model="gpt-4o",
        name="manager",
        system_prompt="당신은 프로젝트 관리자입니다. 팀을 조정하고 결과를 종합합니다.",
    )

    # 작업자 Agent들
    workers = {
        f"worker_{i}": Agent(
            model="gpt-4o",
            name=f"worker_{i}",
            system_prompt=f"당신은 작업자 {i+1}입니다. 주어진 작업을 수행합니다.",
        )
        for i in range(3)
    }

    # 모든 Agent 포함
    all_agents = {"manager": manager, **workers}

    coordinator = MultiAgentCoordinator(agents=all_agents)

    print("📊 계층적 구조:")
    print("  - Manager: 프로젝트 조정")
    print("  - Workers: 실제 작업 수행")

    # 계층적 실행 예제
    # result = await coordinator.execute_hierarchical(
    #     task="프로젝트를 수행하세요.",
    #     manager="manager",
    #     workers=list(workers.keys())
    # )


# 실행 (주석 해제)
# await hierarchical_structure()
print("💡 위의 주석을 해제하여 실행하세요.")

## 5. 에이전트 간 통신

Agent들은 메시지 버스를 통해 통신할 수 있습니다.

### 5.1 메시지 전달

Agent 간 메시지를 주고받을 수 있습니다.


In [ ]:
from beanllm import AgentMessage, MessageType, CommunicationBus


async def message_passing():
    """메시지 전달 예제"""

    # 통신 버스 생성
    bus = CommunicationBus()

    # Agent 생성
    agent1 = Agent(model="gpt-4o", name="agent1")
    agent2 = Agent(model="gpt-4o", name="agent2")

    # 메시지 구독
    def agent1_handler(message: AgentMessage):
        print(f"Agent1이 메시지 수신: {message.content}")

    def agent2_handler(message: AgentMessage):
        print(f"Agent2가 메시지 수신: {message.content}")

    bus.subscribe("agent1", agent1_handler)
    bus.subscribe("agent2", agent2_handler)

    # 메시지 전송
    message = AgentMessage(
        sender="agent1",
        receiver="agent2",
        message_type=MessageType.INFORM,
        content="작업이 완료되었습니다.",
    )

    bus.publish(message)

    print("✅ 메시지 전달 완료!")


# 실행 (주석 해제)
# await message_passing()
print("💡 위의 주석을 해제하여 실행하세요.")

## 6. 실전 예제

### 6.1 종합 리포트 작성 시스템

여러 팀이 협업하여 리포트를 작성하는 시스템입니다.


In [ ]:
async def comprehensive_report_system():
    """종합 리포트 작성 시스템"""

    # 프로젝트 관리자
    pm = Agent(
        model="gpt-4o",
        name="pm",
        system_prompt="당신은 프로젝트 관리자입니다. 팀을 조정하고 결과를 종합합니다.",
    )

    # 연구 팀 (병렬)
    research_team = {
        "data_researcher": Agent(
            model="gpt-4o",
            name="data_researcher",
            system_prompt="관련 데이터와 통계를 찾아 요약합니다.",
        ),
        "lit_researcher": Agent(
            model="gpt-4o",
            name="lit_researcher",
            system_prompt="관련 학술 논문과 주요 발견을 찾습니다.",
        ),
        "market_researcher": Agent(
            model="gpt-4o",
            name="market_researcher",
            system_prompt="시장 동향과 경쟁 환경을 분석합니다.",
        ),
    }

    # 분석 팀 (순차)
    analysis_team = {
        "data_analyst": Agent(
            model="gpt-4o",
            name="data_analyst",
            system_prompt="데이터 패턴을 분석하고 인사이트를 추출합니다.",
        ),
        "statistician": Agent(
            model="gpt-4o",
            name="statistician",
            system_prompt="통계 분석을 수행하고 결과를 검증합니다.",
        ),
    }

    # 모든 Agent
    all_agents = {"pm": pm, **research_team, **analysis_team}

    coordinator = MultiAgentCoordinator(agents=all_agents)

    topic = "2025년 AI가 직장에 미치는 영향"

    print(f"프로젝트 시작: {topic}")
    print("=" * 60)

    # Phase 1: 연구 (병렬)
    print("\n[Phase 1] 연구 팀 (병렬 실행)...")
    research_result = await coordinator.execute_parallel(
        task=f"{topic}에 대해 연구하세요.",
        agents=list(research_team.keys()),
        aggregation="merge",  # 결과 병합
    )

    print(f"✅ 연구 완료: {len(research_result.get('all_answers', []))}개 결과")

    # Phase 2: 분석 (순차)
    print("\n[Phase 2] 분석 팀 (순차 실행)...")
    analysis_result = await coordinator.execute_sequential(
        task=f"다음 연구 결과를 분석하세요:\n{research_result.get('final_result', '')}",
        agent_order=list(analysis_team.keys()),
    )

    print(f"✅ 분석 완료!")
    print(f"\n📊 최종 리포트: {analysis_result.get('final_result', '')[:300]}...")


# 실행 (주석 해제)
# await comprehensive_report_system()
print("💡 위의 주석을 해제하여 실행하세요.")

## 7. 다음 단계

Multi-Agent를 마스터했습니다! 다음 튜토리얼로 진행하세요:

### 📚 추천 학습 순서

1. **[08_vision_rag.ipynb](08_vision_rag.ipynb)** - Vision RAG
   - 이미지 기반 RAG
   - 멀티모달 검색

2. **[09_ocr.ipynb](09_ocr.ipynb)** - OCR
   - 이미지에서 텍스트 추출
   - 문서 디지털화

3. **[10_audio_processing.ipynb](10_audio_processing.ipynb)** - Audio Processing
   - 음성 인식 및 합성
   - 오디오 처리

### 🔗 관련 문서

- [API Reference](../API_REFERENCE.md#multi-agent) - Multi-Agent API 상세 문서
- [README.md](../../README.md) - 프로젝트 개요

---

**✅ Multi-Agent 완료! 다음 노트북으로 진행하세요.**
